In [1]:
from pathlib import Path
from bids import BIDSLayout
from json import loads, dumps

In [2]:
DATA_PATH = Path("/data/datasets/hcph")

In [3]:
layout = BIDSLayout(DATA_PATH, database_path=DATA_PATH / ".bids-index")

In [4]:
entities = {"suffix": "dwi", "direction": "PA", "extension": "json"}
entities = {"suffix": "bold", "task": "rest", "extension": "json", "acquisition": None}
entities = {"suffix": "magnitude1", "extension": "json"}
# entities = {"suffix": "magnitude2", "extension": "json"}
# entities = {"suffix": "phasediff", "extension": "json"}
entities = {"suffix": "epi", "acquisition": "b0", "extension": "json"}
entities = {"suffix": "epi", "acquisition": "bold", "extension": "json"}

In [5]:
if entities["suffix"] == "dwi":
    out_file = DATA_PATH / f"dir-{entities['direction']}_{entities['suffix']}.json"
elif entities["suffix"] == "bold":
    out_file = DATA_PATH / f"task-{entities['task']}_{entities['suffix']}.json"
elif entities["suffix"] == "epi":
    out_file = DATA_PATH / f"acq-{entities['acquisition']}_{entities['suffix']}.json"
else:
    out_file = DATA_PATH / f"{entities['suffix']}.json"

out_dict = loads(out_file.read_text()) if out_file.exists() else None
first_update = False

for meta_path in layout.get(**entities):
    if Path(meta_path.path) == out_file:
        continue

    next_dict = loads(Path(meta_path.path).read_text())    
    next_dict.pop("TaskName", None)
    next_dict.pop("ProcedureStepDescription", None)

    if out_dict is None:
        out_dict = next_dict.copy()
        continue
    elif not first_update:
        next_dict.update(out_dict)
        out_dict = next_dict.copy()
        first_update = True
        continue
        
    for key, value in next_dict.items():
        if key not in out_dict or out_dict.get(key, None) != value:
            out_dict.pop(key, None)

In [6]:
if entities["suffix"] == "dwi":
    deltas = {"PA": (0.0233, 0.0475), "LR": (0.0272, 0.0469)}
    deltas["AP"] = deltas["PA"]
    deltas["RL"] = deltas["LR"]
    out_dict["Deltas"] = deltas[entities["direction"]]

out_dict

{'AcquisitionNumber': 1,
 'BodyPartExamined': 'BRAIN',
 'CoilCombinationMethod': 'Sum of Squares',
 'CoilString': 'HeadNeck_64',
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20220720',
 'DeviceSerialNumber': '67014',
 'EchoTime': 0.035,
 'EchoTime1': 0.035,
 'EchoTime2': 0.001,
 'HeudiconvVersion': '1.0.0',
 'ImagingFrequency': 123.248,
 'InstitutionAddress': 'Rue du Bugnon 21,Lausanne,District,CH,1011',
 'InstitutionName': 'CHUV',
 'MRAcquisitionType': '2D',
 'MTState': False,
 'MagneticFieldStrength': 3,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'MAGNETOM Prisma Fit',
 'MatrixCoilMode': 'GRAPPA',
 'Modality': 'MR',
 'NonlinearGradientCorrection': True,
 'ParallelAcquisitionTechnique': 'GRAPPA',
 'ParallelReductionFactorInPlane': 2,
 'PartialFourier': 0.75,
 'PartialFourierDirection': 'PHASE',
 'PatientPosition': 'HFS',
 'PercentSampling': 100,
 'PhaseResolution': 1,
 'PulseSequenceDetails': '%CustomerSeq%\\cmrr_mbep2d_bold',
 'ReceiveCoilName': 

In [7]:
out_keys = set(out_dict.keys())
for meta_path in layout.get(**entities):
    path = Path(meta_path.path)
    next_dict = loads(path.read_text())
    for key in out_keys:
        next_dict.pop(key, None)
    
    path.write_text(dumps(next_dict, indent=2, sort_keys=True))

In [8]:
out_file.write_text(dumps(out_dict, indent=2, sort_keys=True))

1172

In [9]:
if entities["suffix"] == "bold":
    entities["part"] = "mag"
    
    for meta_path in layout.get(**entities):
        mag_path = Path(meta_path.path)
        phase_path = Path(str(meta_path.path).replace("_part-mag_", "_part-phase_"))
        mag_dict = loads(mag_path.read_text())
        phase_dict = loads(phase_path.read_text())
        
        for key in mag_dict:
            phase_dict.pop(key, None)
        
        if "EchoTime1" in phase_dict:
            mag_dict["PhaseEchoTime1"] = phase_dict.pop("EchoTime1")

        if "EchoTime2" in phase_dict:
            mag_dict["PhaseEchoTime2"] = phase_dict.pop("EchoTime2")
        
        merged_path = Path(str(meta_path.path).replace("part-mag_", ""))
        merged_path.write_text(dumps(mag_dict, indent=2, sort_keys=True))
        
        mag_path.unlink()
        if not phase_dict:
            phase_path.unlink()
elif entities["suffix"] == "epi":
    
    if entities["acquisition"] == "bold":
        entities["part"] = "mag"

    for meta in layout.get(**entities):
        epitype = "dwi" if  entities["acquisition"] == "b0" else "func"
        mag_path = Path(meta.path)
        phase_path = Path(str(mag_path).replace("_part-mag_", "_part-phase_"))
        
        mag_dict = loads(mag_path.read_text())
        phase_dict = loads(phase_path.read_text())
        
        for key in mag_dict:
            phase_dict.pop(key, None)
        
        if "EchoTime1" in phase_dict:
            mag_dict["PhaseEchoTime1"] = phase_dict.pop("EchoTime1")

        if "EchoTime2" in phase_dict:
            mag_dict["PhaseEchoTime2"] = phase_dict.pop("EchoTime2")
        
    
        mag_dict["B0FieldIdentifier"] = f"epi{epitype}{meta.entities['session']}"
        merged_path = Path(str(mag_path).replace("part-mag_", ""))
        merged_path.write_text(dumps(mag_dict, indent=2, sort_keys=True))
        
        mag_path.unlink()
        if not phase_dict:
            phase_path.unlink()
        else:
            phase_path.write_text(dumps(phase_path, indent=2, sort_keys=True))

elif entities["suffix"] in ("magnitude1", "magnitude2", "phasediff"):
    entities["suffix"] = ("magnitude1", "magnitude2", "phasediff")
    for meta in layout.get(**entities):
        meta_path = Path(meta.path)
        meta_dict = loads(meta_path.read_text())
        meta_dict["B0FieldIdentifier"] = f"gre{meta.entities['session']}"
        meta_path.write_text(dumps(meta_dict, indent=2, sort_keys=True))

In [ ]:
entities["suffix"] = "phasediff"

In [ ]:
d.entities["session"]